# 📊 Fundamentos de métricas y evaluación de usuarios


## 🎯 Objetivo de la sesión

* Comprender las métricas clave para evaluar el desempeño del negocio y el comportamiento de los usuarios en productos digitales, y aplicarlas en  ejemplos prácticos.


## Medir para mejorar

Medir es el primer paso para comprender y mejorar. El análisis de datos permite transformar la información en métricas de negocio que revelan patrones, oportunidades y riesgos. Estas métricas hacen posible decisiones *data driven*, donde la intuición se complementa con evidencia, fortaleciendo el rol del profesional de datos como un agente clave en la estrategia empresarial.

*Las métricas de negocio* son indicadores cuantitativos que permiten evaluar el desempeño de una empresa en diferentes áreas: ventas, retención, satisfacción, eficiencia, entre otras.  

Cuando estas métricas se alinean con los objetivos estratégicos se convierten en *KPIs (Key Performance Indicators)*, es decir, métricas clave que guían la toma de decisiones y permiten medir el progreso hacia metas específicas.  

> En resumen: todas las KPIs son métricas, pero no todas las métricas llegan a ser KPIs.


Antes de continuar, realizaremos un análisis de negocio revisando algunos conceptos y métricas de interés. Lo haremos a través de un ejercicio práctico que nos permitirá comprender cómo estas métricas se calculan y aplican, conectando la teoría con la práctica para obtener una visión más clara del desempeño empresarial.

### 🧪 Análisis de resultados de campañas de marketing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
sns.set(style="whitegrid", palette="pastel")



> Eres parte del equipo de análisis de una empresa digital que ha implementado varias campañas de marketing para atraer usuarios y generar conversiones en su plataforma. Ahora cuentas con tres conjuntos de datos clave que te permitirán realizar un análisis integral del desempeño de las campañas:


1. **Interacciones y transacciones `orders`:**  
   - Cada registro representa una interacción de usuario que resultó en una transacción.
   - Columnas: `uuid_interaction` (ID de interacción), `transaction_ts` (fecha/hora), `items` (artículos comprados), `amount` (monto total).

2. **Costos de campañas `campaign_detaials` :**  
   - Información de costos fijos y variables para cada campaña.
   - Columnas: `id_campaña`, `costo_base_campaña`, `costo_por_dia_activo_campaña`, `costo_por_interaccion_campaña`.

3. **Visitas y conversiones de usuarios `visits`:**  
   - Cada registro representa una visita de usuario a la plataforma, asociada a una campaña.
   - Columnas: `uuid_user` (ID de usuario), `ts_visits` (fecha/hora), `campaign` (ID de campaña), `uuid_interaction` (ID de interacción), `convertion` (1 si hubo conversión, 0 si no).

### Sugerencias de análisis de interés

- **Tasa de conversión por campaña:** Calcula el porcentaje de visitas que terminan en conversión para cada campaña.
- **Ingresos y artículos vendidos por campaña:** Suma el monto total y la cantidad de artículos vendidos asociados a cada campaña.
- **Rentabilidad de campañas:** Relaciona los ingresos y conversiones generados con los costos totales (base, por día y por interacción) para identificar campañas más rentables.
- **Comportamiento de usuarios:** Analiza cuántos usuarios únicos visitan, interactúan y convierten en cada campaña.
- **Tendencias temporales:** Visualiza cómo varían las visitas, conversiones e ingresos a lo largo del tiempo y detecta patrones estacionales o picos de actividad.
- **Eficiencia de inversión:** Calcula métricas como el costo por conversión, el ingreso por visita y el retorno de inversión (ROI) para cada campaña.

Con estos análisis podrás obtener una visión completa del desempeño de las campañas, identificar oportunidades de mejora y tomar decisiones informadas para optimizar futuras estrategias

#### Carga de datos y exploración

In [ ]:
visits =pd.read_csv("https://raw.githubusercontent.com/zyntonyson/bootcamp_ds_da/refs/heads/main/datasets/visits.csv")


In [ ]:
orders =pd.read_csv("https://raw.githubusercontent.com/zyntonyson/bootcamp_ds_da/refs/heads/main/datasets/orders.csv")


In [ ]:
campaigns_details =pd.read_csv("https://raw.githubusercontent.com/zyntonyson/bootcamp_ds_da/refs/heads/main/datasets/campaigns_details.csv")

#### `Visits` EDA e ingenieria de características

In [ ]:
visits.info()


In [ ]:
visits.head()

In [ ]:
visits.describe(include='all')

In [ ]:
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Creamos un nuevo DataFrame
visits_enrichment = visits.copy()

# Convertimos la columna ts_visits a tipo datetime
visits_enrichment['ts_visits'] = pd.to_datetime(visits_enrichment['ts_visits'])

# Extraemos solo la fecha de la visita
visits_enrichment['date_visits'] = visits_enrichment['ts_visits'].dt.date

# Obtenemos el nombre del día de la semana
visits_enrichment['dow_id'] = visits_enrichment['ts_visits'].dt.day_name()

# Convertimos el día de la semana a categórico ordenado (de lunes a domingo)
visits_enrichment['dow'] = pd.Categorical(
    visits_enrichment['dow_id'],
    categories=ordered_days,
    ordered=True
)


In [ ]:
print( f"Users : {visits_enrichment.uuid_user.nunique()}")
print( f"Visits : {visits_enrichment.uuid_interaction.nunique()}")
print( f"Start : {visits_enrichment.ts_visits.min()}")
print( f"End : {visits_enrichment.ts_visits.max()}")

#### **Muestra  visitas y usuarios únicos por fecha**

In [ ]:
df=(
    visits_enrichment
        .groupby(['date_visits'],as_index=False)
        .agg(
            visitas = ('uuid_interaction','nunique'),
            usuarios_unicos = ('uuid_user','nunique'),
        )
        .sort_values(by='date_visits')
)
df.head()

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x='date_visits', y='visitas', label='Visitas')
sns.lineplot(data=df, x='date_visits', y='usuarios_unicos', label='Usuarios Únicos')

plt.title("Usuarios y Visitas durante las campañas")
plt.xlabel("Fecha")
plt.ylabel("Frecuencia")
plt.legend()
plt.tight_layout()
plt.show()

#### **Muestra la conversión diaria por campaña**

In [ ]:
(
    visits_enrichment
        .groupby(['campaign'],as_index=False)
        .agg(
            visitas = ('uuid_interaction','nunique'),
            convertions = ('convertion','sum'),
        )
        .assign(
            rate_convertion_daily = lambda df: (df.convertions/df.visitas).round(3)
        )
        .sort_values(by='rate_convertion_daily',ascending=False)
)

> CCQ: “Si una campaña tuvo 200 visitas y 20 conversiones, ¿cuál es su tasa de conversión?”

In [ ]:
df=(
    visits_enrichment
        .groupby(['date_visits','campaign'],as_index=False)
        .agg(
            visitas = ('uuid_interaction','nunique'),
            convertions = ('convertion','sum'),
        )
        .assign(
            rate_convertion_daily = lambda df: (df.convertions/df.visitas).round(3)
        )
        .sort_values(by='date_visits')
)
df.head()

>

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='date_visits', y='rate_convertion_daily', hue='campaign', marker='o')

plt.title("Tasa de Conversión Diaria por Campaña")
plt.xlabel("Fecha")
plt.ylabel("Tasa de Conversión Diaria")
plt.legend(title="Campaña", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

> Para pensar 🤔 : ¿Cuál creen que es más útil para un negocio: medir visitas o conversiones? ¿Por qué?

#### `Orders` EDA e ingenieria de características

In [ ]:
orders.info()

In [ ]:
orders.head()

In [ ]:
# 1) Hacemos el merge con las columnas necesarias de visits
cols_visits = ['uuid_interaction', 'ts_visits', 'campaign', 'uuid_user']
orders_enrichment = orders.merge(visits[cols_visits], how='inner').copy()

# 2) Conversión de tipos a datetime
orders_enrichment['transaction_ts'] = pd.to_datetime(orders_enrichment['transaction_ts'])  # transacción a datetime
orders_enrichment['ts_visits'] = pd.to_datetime(orders_enrichment['ts_visits'])            # visita a datetime

# 3) Cálculo de días transcurridos hasta la conversión
orders_enrichment['days_elapsed_until_convertion'] = (
    orders_enrichment['transaction_ts'] - orders_enrichment['ts_visits']
).dt.days  # diferencia en días


orders_enrichment


In [ ]:
orders_enrichment.describe()

> CCQ: “¿Qué significa que una campaña tenga en promedio 10 días hasta conversión? ¿Es bueno o malo?”

#### **Muestra los ingresos diarios por campaña**

In [ ]:
# 1) Copia para evitar cambio en los datos originales
df = orders_enrichment.copy()

# 2) Convertir a formato fecha
df['transaction_date'] = df['transaction_ts'].dt.date     # fecha (sin hora) de la transacción
df['campaign'] = df['campaign'].astype(str)                # aseguramos tipo string

# 3) Agregaciones por fecha-campaña
df = df.groupby(['transaction_date', 'campaign'], as_index=False).agg(
    users_converted = ('uuid_user', 'nunique'),              # usuarios únicos que convirtieron
    convertions = ('uuid_interaction', 'nunique'),           # número de conversiones (interacciones únicas)
    revenue = ('amount', 'sum')                              # ingresos totales
)

# 4) Revenue en miles
df['k_revenue'] = df['revenue'] / 1000


df.head()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='transaction_date', y='k_revenue', hue='campaign', marker='o')

# Etiquetas
plt.title("Daily (K) Revenue per Campaign")
plt.xlabel("Transaction Date")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.legend(title="Campaign", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### **Por campaña muestra la cantidad de interacciones, conversiones, ingresos generados, tiempo promedio de conversión**

In [ ]:

tmp = visits_enrichment.copy()

# Agregaciones por campaña
grouped = tmp.groupby(['campaign'], as_index=False).agg(
    visitas=('uuid_interaction', 'nunique'),   # visitas únicas (interacciones únicas)
    convertions=('convertion', 'sum'),         # total de conversiones (suma de flag/columna convertion)
    start_campaign=('ts_visits', 'min'),       # primer timestamp de visita por campaña
    end_campaign=('ts_visits', 'max')          # último timestamp de visita por campaña
)

# Columnas derivadas
grouped['rate_convertion'] = (grouped['convertions'] / grouped['visitas']).round(3)  # tasa de conversión
grouped['campaigns_days_duration'] = (grouped['end_campaign'] - grouped['start_campaign']).dt.days  # duración en días

# Ordenamos por la tasa de conversión
visits_summary = grouped.sort_values(by='rate_convertion', ascending=False)


visits_summary


In [ ]:
tmp = orders_enrichment.copy()

tmp['transaction_date'] = tmp['transaction_ts'].dt.date

#  Agregaciones por campaña
grouped = tmp.groupby(['campaign'], as_index=False).agg(
    users_converted=('uuid_user', 'nunique'),                 # usuarios únicos que convirtieron
    convertions_interact=('uuid_interaction', 'nunique'),     # número de conversiones (interacciones únicas)
    revenue=('amount', 'sum'),                                # ingresos totales
    time_convertion_avg=('days_elapsed_until_convertion', 'mean')  # promedio de días a conversión
)

#  (revenue en miles)
grouped['k_revenue'] = grouped['revenue'] / 1000

#
orders_summary = grouped


In [ ]:
v_cols = ['campaign', 'visitas', 'rate_convertion', 'campaigns_days_duration']
o_cols = ['campaign', 'revenue', 'time_convertion_avg']


# Merge df
campaigns_results = visits_summary[v_cols].merge(orders_summary[o_cols], how='inner', on='campaign')

#    - Ingreso por interacción (visita)
campaigns_results['rate_revenue_interaction'] = (campaigns_results['revenue'] / campaigns_results['visitas']).round(3)

#    - Ingreso por conversión (visita * tasa de conversión)
campaigns_results['rate_revenue_convertion'] = (
    campaigns_results['revenue'] / (campaigns_results['visitas'] * campaigns_results['rate_convertion'])
).round(3)

#
campaigns_results


#### **Muestra el ROMI por campaña**

El **ROMI (Return On Marketing Investment)** mide la rentabilidad de una campaña comparando los ingresos generados con el gasto en marketing.  
El ROMI se enfoca en evaluar el impacto de las inversiones de marketing, mostrando qué campañas realmente aportan valor al negocio.  

La fórmula es:

$$
ROMI = \frac{\text{Ingresos - Inversión en Marketing}}{\text{Inversión en Marketing}}
$$


In [ ]:
campaigns_details.info()

In [ ]:
campaigns_details.head()

In [ ]:
def get_romi(row):
    # Costos individuales
    base = row['costo_base_campaña']
    costo_dias = row['costo_por_dia_activo_campaña'] * row['campaigns_days_duration']
    costo_interacciones = row['costo_por_interaccion_campaña'] * row['visitas']

    # Costo total
    costo_total = base + costo_dias + costo_interacciones

    # Ingreso
    revenue = row['revenue']

    # ROMI
    if costo_total == 0:
        return None  # Venta organica
    return (revenue - costo_total) / costo_total

In [ ]:
(
    campaigns_details
        .rename(columns={'id_campaña':'campaign'})
        .merge(campaigns_results[['visitas','campaigns_days_duration','revenue','campaign']],how='inner',on='campaign')
        .assign(
            ROMI= lambda df: df.apply(get_romi,axis=1)
        )
        .sort_values('ROMI',ascending=False)
)

> CCQ 🤔 : “¿Por qué una campaña con ROMI bajo podría seguir siendo relevante para el negocio?”

## ✅ Para cerrar

---


1. **La integración de datos de visitas, interacciones y costos permite un análisis completo del desempeño de campañas de marketing**, facilitando la identificación de las campañas más rentables y efectivas.

2. **El uso de métricas de negocios** son importantes para entender el estado del negocio ayudando a tomar decisiones informadas sobre inversión y optimización.


> ¿Que potencial ves de lo que hemos tratado hoy?, ¿Cómo se te ocurre usarlo?




## 🚀 Para seguir aprendiendo :

---

- 📚 Vuelve a revisar este notebook junto con la grabación y ¡anímate a ejecutar todo el código por tu cuenta!
- 💬 Recuerda que en Discord puedes dejar todos tus comentarios y dudas sobre el contenido del sprint en [Sprint 9](https://discord.com/channels/1081207584104656986/1270074211540406342).
    - 📝 Si tienes preguntas sobre tu proyecto, usa el canal `#project` para recibir ayuda y compartir ideas.
    - 🤝 Aprovecha el espacio de `CoLearning` para aclarar tus dudas junto con otros estudiantes e instructores: [Co-Learning](https://discord.com/channels/1081207584104656986/1197953851391746119).
- 📅 ¿Necesitas ayuda personalizada? Puedes agendar una sesión `1:1` conmigo aquí: [1:1 Roman Castillo](https://scheduler.zoom.us/roman-castillo/1-1-roman-castillo).

¡Sigue practicando y no dudes en pedir apoyo cuando lo necesites! 💪✨